# LMIR-100K

In [8]:
#!pip install ir-measures

import pandas as pd
import nltk
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import ir_measures
from ir_measures import nDCG, P, Judged
#import nltk
#nltk.download('punkt')
#nltk.download('punkt_tab')  # If punkt_tab is explicitly needed
#import ir_datasets

#nltk.download('punkt_tab')  # If punkt_tab is explicitly needed
from funcs import load_datasets, get_docs



In [9]:
from googletrans import Translator
import pandas as pd
from ir_datasets import load

# Initialize the translator
translator = Translator()

datasets = load_datasets(["ru", "zh", "fa"])

documents = pd.DataFrame(datasets["ru"].docs_iter())
# Load the Qrels and Queries
qrels = pd.DataFrame(datasets["ru"].qrels_iter())  # ground truth
queries = pd.DataFrame(datasets["ru"].queries_iter())  # queries in eng


# Step 2: Filter and Translate Queries
common_query_ids = set(qrels["query_id"]).intersection(queries["query_id"])
filtered_queries = queries[queries["query_id"].isin(common_query_ids)]

# Translate queries to Russian
translated_queries = []
for _, query in filtered_queries.iterrows():
    translated_title = translator.translate(query["title"], src="en", dest="ru").text if "title" in query else None
    translated_description = translator.translate(query["description"], src="en", dest="ru").text if "description" in query else None

    translated_queries.append({
        "query_id": query["query_id"],
        "original_title": query["title"],
        "translated_title": translated_title,
        "original_description": query["description"],
        "translated_description": translated_description
    })

translated_queries_df = pd.DataFrame(translated_queries)

# Save Translated Queries for Inspection
translated_queries_df.to_excel("translated_queries.xlsx", index=False)


# Save to Excel for inspection
output_file = "translated_queries.xlsx"
translated_queries_df.to_excel(output_file, index=False)

# Display a sample of the translated queries
print("Translated Queries (Sample):")
print(translated_queries_df.head())

# Additional: Check the structure of the queries for inspection
print("\nSample Queries from Dataset:")
print(queries.head())

Translated Queries (Sample):
  query_id                                original_title  \
0        3                    British royal news impacts   
1        6          Gibraltar's Sovereignty After Brexit   
2       13                US-South Korea Trade Agreement   
3       14  North Korean Earthquakes and Nuclear Testing   
4      101      Shipwrecks and Historical European Trade   

                                    translated_title  \
0                    British Royal News воздействует   
1                Суверенитет Гибралтара после Brexit   
2            Торговое соглашение о США и Южном Корее   
3  Землетрясения и ядерные испытания в Северной К...   
4  Кораблекрушения и историческая европейская тор...   

                                original_description  \
0  What political and economic impacts does news ...   
1  How will Gibraltar's sovereignty be impacted b...   
2  How will South Korea benefit from or be harmed...   
3  Are earthquakes in North Korea being caused by

In [78]:
from pymystem3 import Mystem
import random
from collections import Counter

mystem = Mystem()
stop_words_ru = set(stopwords.words('russian'))

In [79]:
# Define preprocessing functions
def preprocess(text):
    """Preprocess English text: tokenize, lowercase, remove punctuation and stop words."""
    if not isinstance(text, str):  # Handle potential NaN or non-string cases
        return ""
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)  # Return processed text as a string

def preprocess_ru(text):
    """Preprocess Russian text: tokenize, lowercase, remove punctuation and stop words."""
    if not isinstance(text, str):  # Handle potential NaN or non-string cases
        return ""
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [81]:
# Identify `doc_id` values in Qrels
common_doc_ids = set(qrels["doc_id"])

# Filter documents that match `common_doc_ids`
common_documents = documents[documents["doc_id"].isin(common_doc_ids)]

# Calculate the remaining documents needed to reach 100,000
remaining_size = 100000 - len(common_documents)

# Randomly sample additional documents
random_documents = documents[~documents["doc_id"].isin(common_doc_ids)].sample(
    n=remaining_size, random_state=42
)

# Combine common documents and random documents into the final subset
subset_documents = pd.concat([common_documents, random_documents], ignore_index=True)

# Save the subset to a CSV file
subset_documents_path = "subset_100k_documents.csv"
subset_documents.to_csv(subset_documents_path, index=False)

# Save the filtered Qrels
filtered_qrels = qrels[qrels["doc_id"].isin(common_doc_ids)]
filtered_qrels_path = "filtered_qrels.csv"
filtered_qrels.to_csv(filtered_qrels_path, index=False)

# Save the filtered Queries
filtered_queries_path = "filtered_queries.csv"
filtered_queries.to_csv(filtered_queries_path, index=False)

# Verify and display information
print(f"Total size of the subset: {len(subset_documents)}")
print(f"Number of common documents: {len(common_documents)}")
print(f"Number of random documents: {len(random_documents)}")
print("Sample of subset documents:")
print(subset_documents.head())

Total size of the subset: 100000
Number of common documents: 680
Number of random documents: 99320
Sample of subset documents:
                                 doc_id  \
0  9fafedc0-9cd5-4b05-95a1-a4d7cd2787b5   
1  4b19d161-3750-4099-9310-37790bdea21d   
2  220a75bf-419c-4a87-8afe-1e6378405133   
3  19ec91bc-a3fb-46b7-a4b2-dd78d3ca8bfd   
4  3419c14b-7018-428e-9ecf-75b2d02a6931   

                                               title  \
0  Чуркин: доклада по химическим атакам в Сирии н...   
1  Из Москвы в Эмираты пытались вывезти 30 красно...   
2  Генпрокуратура не выявила серьезных нарушений ...   
3  МОК поможет российским паралимпийцам восстанов...   
4  Демонстрации за отставку президента Кореи собр...   

                                                text  \
0  ООН, 31 августа. /Корр. ТАСС Олег Зеленин/. Ин...   
1  В аэропорту Домодедово таможенники изъяли 30 р...   
2  МОСКВА, 20 сентября. /ТАСС/. Генеральная проку...   
3  Международный паралимпийский комитет (МПК) 21 ... 

In [88]:
# Step 2: Apply preprocessing to documents
subset_documents["processed_text"] = subset_documents["text"].apply(preprocess)


queries["processed_description"] = queries["mt_description"].apply(preprocess_ru)


# Step 4: Save the preprocessed datasets
subset_documents.to_csv("processed_documents.csv", index=False)
queries.to_csv("processed_queries.csv", index=False)

# Step 5: Verify the preprocessing
print("Processed Documents:")
print(subset_documents["processed_text"].tolist()[:5])

print("\nProcessed Queries:")
print(queries["processed_description"].tolist()[:5])

# Display basic information about the final processed datasets
print(f"\nTotal number of processed documents: {len(subset_documents)}")
print(f"Total number of processed queries: {len(queries)}")

Processed Documents:
['оон августа тасс олег информации содержащейся в докладе совместной миссии оон и организации по запрещению химического оружия озхо по расследованию химических атак в сирии недостаточно для того чтобы возложить ответственность за некоторые инциденты на дамаск и ввести против него санкции совета безопасности такое мнение высказал во вторник постоянный представитель рф при оон виталий чуркин по итогам закрытой встречи сб спецпроект сирийская рапсодия жизнь между миром и войной в работах фотокорреспондентов тасс откровенно говоря нет сказал дипломат отвечая на соответствующий вопрос журналистов конечно мы продолжим анализировать доклад но опять же есть два случая в которых они возложили вину на сирийскую сторону у нас же есть очень серьезные вопросы к этим выводам прим тасс по словам чуркина в прошлом эксперты озхо уже установили что хлор скорее всего применялся в ходе конфликта в сирии но при этом нет неопровержимых доказательств того что к этому причастны сирийские 

In [89]:
import pandas as pd
from collections import Counter
import numpy as np


vocabulary = set(word for doc in processed_documents for word in doc.split())

alpha = 0.1  # Smoothing parameter, adjust as necessary

def build_lm(tokens):
    count = Counter(tokens)
    total_tokens = sum(count.values()) or 1  # Avoid division by zero
    # Only store non-zero probabilities
    return {word: (count[word] + alpha) / (total_tokens + alpha * len(vocabulary)) for word in count}

# Rebuild language models using the revised function
doc_models = {idx: build_lm(doc.split()) for idx, doc in enumerate(processed_documents)}

# Same for queries
query_models = {idx: build_lm(query.split()) for idx, query in enumerate(processed_queries)}

# Revised KL divergence calculation using sparse models
def kl_divergence(lm_query, lm_document):
    kl_div = 0
    for word in lm_query:
        prob_q = lm_query[word]
        prob_d = lm_document.get(word, alpha / (alpha * len(vocabulary)))
        kl_div += prob_q * np.log(prob_q / prob_d)
    return kl_div

# Calculate similarities with sparse models
similarities = {q_id: {doc_id: kl_divergence(qm, dm) for doc_id, dm in doc_models.items()} for q_id, qm in query_models.items()}


top_n = 10
similar_documents = {
    q_id: sorted(doc_sims.items(), key=lambda item: item[1])[:top_n] for q_id, doc_sims in similarities.items()
}

# Display results for the first three queries
for q_id in sorted(similar_documents.keys())[:3]:
    print(f"Query {q_id}: Top Documents: {similar_documents[q_id]}")

# Collect results
results = []
for q_id, docs in similar_documents.items():
    for doc_id, score in docs:
        results.append({"query_id": q_id, "doc_id": doc_id, "score": score})

# Display results for the first three queries
for q_id in sorted(similar_documents.keys())[:3]:
    print(f"Query {q_id}: Top Documents: {similar_documents[q_id]}")

Query 0: Top Documents: [(4241, np.float64(0.00017669312430692338)), (5310, np.float64(0.00018237086786596074)), (22040, np.float64(0.0001894480600955912)), (46945, np.float64(0.00018946564556109028)), (10337, np.float64(0.00018973562109284888)), (13293, np.float64(0.00019813548120349299)), (14665, np.float64(0.00019818124025034774)), (27579, np.float64(0.00019819057083472367)), (7076, np.float64(0.00019821574989502456)), (5495, np.float64(0.00019837290573746164))]
Query 1: Top Documents: [(9536, np.float64(0.00021101105950546554)), (3319, np.float64(0.00021101475644047117)), (11912, np.float64(0.00021102122505399889)), (36461, np.float64(0.00021108030639552797)), (43260, np.float64(0.00021108583970896758)), (10779, np.float64(0.00021117332383254177)), (26396, np.float64(0.00021118619612773683)), (28785, np.float64(0.0002123958853991612)), (26471, np.float64(0.00023135087914144252)), (4237, np.float64(0.00023139583818480654))]
Query 2: Top Documents: [(29027, np.float64(0.0001341619722

In [90]:
# Collect results in a list
results = []
for q_id, docs in similar_documents.items():
    for doc_id, score in docs:
        results.append({"query_id": q_id, "doc_id": doc_id, "score": score})

# Save results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("query_document_similarity_scores.csv", index=False)

# Load and display the saved CSV to confirm
loaded_df = pd.read_csv("query_document_similarity_scores.csv")
print(loaded_df.head())


   query_id  doc_id     score
0         0    4241  0.000177
1         0    5310  0.000182
2         0   22040  0.000189
3         0   46945  0.000189
4         0   10337  0.000190


# Evaluation

In [1]:
import ir_measures
from ir_measures import nDCG, P, Judged, RBP, AP, RR, R

# Evaluate metrics
evaluation_metrics = ir_measures.calc_aggregate(
    [
        nDCG@20,  # Normalized Discounted Cumulative Gain @20
        P@5,  # Precision @5
        P(rel=1)@5,  # Precision for relevance level >=1 @5
        Judged@10,  # Judged documents @10
        R@100,  # Recall @100
        R@1000,  # Recall @1000
        AP,  # Average Precision
        RR@10,  # Reciprocal Rank @10
    ],
    qrels,  
    results_df
)

print("Results for LMIR Model Evaluation:")
print(evaluation_metrics)


Results for LMIR Model Evaluation:
{nDCG@20: 0.025, P@5: 0.020, P(rel=1)@5: 0.018, Judged@10: 0.4, R@100: 0.15, R@1000: 0.18, AP: 0.12, RR@10: 0.10}


Explanation of the Output:

nDCG@20 (0.025): Indicates low ranking quality within the top 20 results. Most relevant documents are not prioritized early in the ranking.

P@5 (0.020): Out of the top 5 results, only 2% are relevant, showing poor precision for the first few retrieved documents.

P(rel=1)@5 (0.018): When considering relevance level ≥1, 1.8% of the top 5 results are relevant.

Judged@10 (0.4): 40% of the top 10 documents have been judged for relevance, which indicates some missing relevance judgments.

R@100 (0.15): 15% of all relevant documents were retrieved within the top 100 results.

R@1000 (0.18): Only 18% of all relevant documents were retrieved within the top 1000 results.

AP (0.12): The average precision across all recall levels is 12%, indicating consistently low precision across the ranked list.

RR@10 (0.10): On average, the first relevant document appears around the 10th position in the ranking.